In [126]:
from copy import copy
from dataclasses import dataclass
from typing import Any, List


def evaluate(a):
    try:
        return copy(a.eval())
    except AttributeError:
        return copy(a)


def validate(a):
    if not (a in [0, 1] or isinstance(a, (str, And, Or, Xor, Not))):
        raise ValueError(
            "Value must be 0, 1, or an instance of str, And, Or, Xor, or Not."
        )
    else:
        return copy(a)


@dataclass
class And:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (0, _) | (_, 0):
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 0
            case (1, a) | (a, 1):
                return a
            case _:
                return self

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}&{self.b})"


@dataclass
class Or:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (1, _) | (_, 1):
                return 1
            case (0, a) | (a, 0):
                return a
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case _:
                return self

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}|{self.b})"


@dataclass
class Xor:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (0, 1) | (1, 0):
                return 1
            case (0, 0) | (1, 1):
                return 0
            case (0, a) | (a, 0):
                return a
            case (1, a) | (a, 1):
                return Not(a)
            case (a, b) if a == b:
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case (Xor(a, b), c) | (a, Xor(b, c)) if a == b:
                return c
            case (Xor(a, b), c) | (a, Xor(b, c)) if a == c:
                return b
            case (Xor(a, b), c) | (a, Xor(b, c)) if b == c:
                return a
            case _:
                return copy(self)

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}^{self.b})"


@dataclass
class Not:
    a: Any

    def __init__(self, a):
        self.a = validate(a)

    def eval(self):
        match evaluate(self.a):
            case 0:
                return 1
            case 1:
                return 0
            case Not(a):
                return a
            case _:
                return copy(self)

    def __str__(self):
        r = self.eval()
        if r in [0, 1]:
            return f"{r}"
        return f"~{self.a}"

In [127]:
# Example usage:
assert And("a", 0).eval() == 0
assert And("a", 1).eval() == "a"
assert And("a", Not("a")).eval() == 0
assert Or("a", 1).eval() == 1
assert Or("a", 0).eval() == "a"
assert Or("a", Not("a")).eval() == 1
assert Xor("a", 1).eval() == Not("a")
assert Xor("a", 0).eval() == "a"
assert Xor("a", Not("a")).eval() == 1
assert Xor("a", "a").eval() == 0
assert Xor("a", Xor("a", "b")).eval() == "b"
assert Not(Not("a")).eval() == "a"

In [128]:
@dataclass
class BitArray:
    bits: List

    def from_int(n, length):
        """Create a BitArray from an integer"""
        return BitArray([int(i) for i in bin(n)[2:].zfill(length)])
    
    def to_int(self):
        """Converts the BitArray to an integer. If the BitArray contains bits with unknown values, returns None."""
        if not all(isinstance(evaluate(bit), int) for bit in self.bits):
            return None
        return int("".join(str(bit) for bit in self.bits), 2)

    def len(self):
        """Returns the length of the BitArray"""
        return len(self.bits)

    def __and__(self, other):
        """Bitwise AND"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([And(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __or__(self, other):
        """Bitwise OR"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([Or(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __xor__(self, other):
        """Bitwise XOR"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([Xor(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __invert__(self):
        """Bitwise NOT"""
        return BitArray([Not(a) for a in self.bits])

    def __iand__(self, other):
        """In-place bitwise AND"""
        return self & other

    def __ior__(self, other):
        """In-place bitwise OR"""
        return self | other

    def __ixor__(self, other):
        """In-place bitwise XOR"""
        return self ^ other

    def __lshift__(self, n):
        """Shift left by n bits"""
        shifted = self.bits[n:] + [0] * n
        return BitArray(shifted)

    def __rshift__(self, n):
        """Shift right by n bits"""
        shifted = [0] * n + self.bits[:self.len() - n]
        return BitArray(shifted)

    def __str__(self):
        return "_".join(str(bit) for bit in self.bits)

In [129]:
# Example usage:
a = BitArray(["a", "b", "c", "d"])
b = BitArray.from_int(0b1100, 4)

assert f"{a}" == "a_b_c_d"
assert f"{b}" == "1_1_0_0"
assert b.to_int() == 0b1100
assert f"{a & b}" == "a_b_0_0"
assert f"{(a | b)}" == "1_1_c_d"
assert f"{(a ^ b)}" == "~a_~b_c_d"
assert f"{(~a)}" == "~a_~b_~c_~d"
assert f"{(a << 2)}" == "c_d_0_0"
assert f"{(a >> 2)}" == "0_0_a_b"

In [130]:
@dataclass
class U64:
    x: int

    def __init__(self, x):
        self.x = x & (2**64 - 1)

    def __and__(self, other):
        return U64(self.x & other.x)

    def __or__(self, other):
        return U64(self.x | other.x)

    def __xor__(self, other):
        return U64(self.x ^ other.x)

    def __invert__(self):
        return U64(~self.x & (2**64 - 1))

    def __lshift__(self, n):
        return U64(self.x << n)

    def __rshift__(self, n):
        return U64(self.x >> n)

    def __str__(self):
        return f"{self.x:#0{18}x}"

    def __int__(self):
        return self.x

    def rand():
        from random import randint

        return U64(randint(0, 2**64 - 1))

    def rotate_left(self, n):
        return U64((self.x << n) | (self.x >> (64 - n)))

    def rotate_right(self, n):
        return U64((self.x >> n) | (self.x << (64 - n)))


# Randomized tests for BitArray with 64 bits
count = 100
for i in range(count):
    a = U64.rand()
    b = U64.rand()

    arr_a = BitArray.from_int(int(a), 64)
    arr_b = BitArray.from_int(int(b), 64)

    assert int(a & b) == (arr_a & arr_b).to_int()
    assert int(a | b) == (arr_a | arr_b).to_int()
    assert int(a ^ b) == (arr_a ^ arr_b).to_int()
    assert int(~a) == (~arr_a).to_int()
    assert int(a << 2) == (arr_a << 2).to_int()
    assert int(a >> 2) == (arr_a >> 2).to_int()
    print(f"{i+1} of {count} tests passed.", end="\r")

In [131]:
# Test transposing an 8x8 bit matrix stored in a 64-bit integer:

def permute_step(x, mask, shift):
    t = ((x >> shift) ^ x) & mask
    return (x ^ t) ^ (t << shift)


matrix = BitArray([f"a{i:02}" for i in range(64)])
m0 = BitArray.from_int(0x00AA00AA00AA00AA, 64)
m1 = BitArray.from_int(0x0000CCCC0000CCCC, 64)
m2 = BitArray.from_int(0x00000000F0F0F0F0, 64)

assert matrix.len() == 64
assert m0.len() == 64
assert m1.len() == 64
assert m2.len() == 64

matrix = permute_step(matrix, m0, 7)
matrix = permute_step(matrix, m1, 14)
matrix = permute_step(matrix, m2, 28)

# fmt: off
transpose = [
    "a00", "a08", "a16", "a24", "a32", "a40", "a48", "a56",
    "a01", "a09", "a17", "a25", "a33", "a41", "a49", "a57", 
    "a02", "a10", "a18", "a26", "a34", "a42", "a50", "a58", 
    "a03", "a11", "a19", "a27", "a35", "a43", "a51", "a59",
    "a04", "a12", "a20", "a28", "a36", "a44", "a52", "a60",
    "a05", "a13", "a21", "a29", "a37", "a45", "a53", "a61",
    "a06", "a14", "a22", "a30", "a38", "a46", "a54", "a62",
    "a07", "a15", "a23", "a31", "a39", "a47", "a55", "a63",
]

assert matrix == BitArray(transpose)